In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

In [2]:
examples = [
    {
        "movie": "Inception",
        "answer": """
          영화 제목: 인셉션 (Inception)
          장르: "SF (Science Fiction)",
          평점: 8.8,
          개봉 연도: 2010,
          줄거리: "꿈을 조작하는 도둑이 꿈의 세계에서 임무를 수행하며 현실과 꿈의 경계를 넘나드는 이야기.",
          감독: "크리스토퍼 놀란 (Christopher Nolan)",
          출연진: ["레오나르도 디카프리오 (Leonardo DiCaprio)", "조셉 고든 레빗(Joseph Gordon-Levitt)", "엘리엇 페이지 (Elliot Page)"],
          정보: "이 영화는 꿈의 구조와 현실의 경계를 탐구하며, 시각적으로 혁신적인 장면들로 유명합니다.",
          예산: "$160 million",
          박스 오피스: "$836.8 million worldwide",
        """
    },
    {
      "movie": "인터스텔라",
      "answer": """
        영화 제목: 인터스텔라 (Interstellar)
        장르: "SF (Science Fiction)",
        평점: 8.6,
        개봉 연도: 2014,
        줄거리: "지구의 생존을 위해 우주로 떠나는 탐험대의 이야기로, 시간과 공간을 넘나드는 모험을 그립니다.",
        감독: "크리스토퍼 놀란 (Christopher Nolan)",
        출연진: ["매튜 맥커너히 (Matthew McConaughey)", "앤 해서웨이 (Anne Hathaway)", "제시카 차스테인 (Jessica Chastain)"],
        정보: "이 영화는 상대성 이론과 블랙홀을 다루며, 놀라운 시각 효과와 감동적인 스토리로 많은 찬사를 받았습니다.",
        예산: "$165 million",
        박스 오피스: "$677.5 million worldwide",
      """
    },
    {
      "movie": "데드풀",
      "answer": """
        영화 제목: 데드풀 (Deadpool)
        장르: "액션, 코미디",
        평점: 8.0,
        개봉 연도: 2016,
        줄거리: "암살자이자 슈퍼히어로인 데드풀이 자신의 정체성을 찾고 복수를 위해 싸우는 이야기.",
        감독: "팀 밀러 (Tim Miller)",
        출연진: ["라이언 레이놀즈 (Ryan Reynolds)", "모레나 바카린 (Morena Baccarin)", "T.J. 밀러 (T.J. Miller)"],
        정보: "이 영화는 유머와 액션을 결합하여 슈퍼히어로 장르에 새로운 바람을 불어넣었습니다.",
        예산: "$58 million",
        박스 오피스: "$783.1 million worldwide",
      """
    }
]

example_template = """
  영화: {movie}
  답변: {answer}
  """

example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
  examples=examples,
  example_prompt=example_prompt,
  suffix="{movie}",
  input_variables=["movie"]
)

chain = prompt | chat

chain.invoke({"movie": "아바타"})


  영화: 아바타
  답변: 
        영화 제목: 아바타 (Avatar)
        장르: "SF (Science Fiction), 액션, 모험",
        평점: 7.8,
        개봉 연도: 2009,
        줄거리: "인류의 탐욕으로부터 자연을 지키기 위해 싸우는 외계 행성 판도라의 이야기.",
        감독: "제임스 카메론 (James Cameron)",
        출연진: ["샘 워싱턴 (Sam Worthington)", "조디 포스터 (Zoe Saldana)", "시고니 위버 (Sigourney Weaver)"],
        정보: "이 영화는 혁신적인 3D 기술과 환경 보호주의 메시지로 유명하며, 세계적으로 대단한 흥행을 거뒀습니다.",
        예산: "$237 million",
        박스 오피스: "$2.9 billion worldwide",
      
  

AIMessageChunk(content='\n  영화: 아바타\n  답변: \n        영화 제목: 아바타 (Avatar)\n        장르: "SF (Science Fiction), 액션, 모험",\n        평점: 7.8,\n        개봉 연도: 2009,\n        줄거리: "인류의 탐욕으로부터 자연을 지키기 위해 싸우는 외계 행성 판도라의 이야기.",\n        감독: "제임스 카메론 (James Cameron)",\n        출연진: ["샘 워싱턴 (Sam Worthington)", "조디 포스터 (Zoe Saldana)", "시고니 위버 (Sigourney Weaver)"],\n        정보: "이 영화는 혁신적인 3D 기술과 환경 보호주의 메시지로 유명하며, 세계적으로 대단한 흥행을 거뒀습니다.",\n        예산: "$237 million",\n        박스 오피스: "$2.9 billion worldwide",\n      \n  ')